In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
import bisect
import sys
from pickle import FALSE
from collections import OrderedDict


sys.path.append('..')  # Add the parent directory to the sys.path

import utils.dataset_loader as dataset_loader

! gpustat

rambo6                      Fri May 31 23:05:42 2024  525.147.05
[0] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     3 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     3 / 24576 MB |
[2] NVIDIA GeForce RTX 3090 | 26°C,   0 % |     3 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 23°C,   0 % |     3 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     3 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 24°C,   0 % |     3 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 48°C,   0 % |   375 / 24576 MB | liranc6(372M)
[7] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     3 / 24576 MB |


In [3]:
# split the windows to fixed size context and label windows
fs = 250
context_window_size = 9*60*fs  # minutes * seconds * fs
label_window_size = 1*60*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size


split_pSignal_file = '/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5'
# '/mnt/qnap/liranc6/data/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5'

# Instantiate the class
dataset = dataset_loader.SingleLeadECGDatasetCrops(context_window_size, label_window_size, split_pSignal_file, start_patiant=3, end_patiant=6, data_with_RR=True, return_with_RR=True)

# Loop over the dataset
# print(f'{len(dataset)=}')
for i in range(len(dataset)):
    # print(i)
    context, label = dataset.__getitem__(i)
    print(f"{context.shape=}, {label.shape=}")
    # Perform any assertions or checks here
    if i >9:
        break

key='00003'
key='00004'
key='00005'
key='00006'
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
